In [6]:

import pandas as pd
from qmt_data import QmtData
import os
import datetime

In [7]:
today = datetime.datetime.today().strftime("%Y%m%d")

In [8]:
qmt_data = QmtData()
ticker_df = qmt_data.get_tickers()
stock_list = ticker_df["ticker"].tolist()
start_time = today
end_time = today
period = "1m"
qmt_data.download_data(
    start_time=start_time, end_time=end_time, period=period, stock_list=stock_list
)
record = qmt_data.write_to_ftr_parallel(
    stock_list=stock_list,
    start_time=start_time,
    end_time=end_time,
    period=period,
)
record_df = pd.DataFrame(record)


  0%|          | 4/7853 [00:00<03:27, 37.74it/s]

Downloaded data for 601882.SH
Downloaded data for 603995.SH
Downloaded data for 688720.SH
Downloaded data for 601128.SH
Downloaded data for 600740.SH
Downloaded data for 603909.SH
Downloaded data for 600734.SH
Downloaded data for 600642.SH


  0%|          | 14/7853 [00:00<03:06, 41.92it/s]

Downloaded data for 600259.SH
Downloaded data for 605298.SH
Downloaded data for 601968.SH
Downloaded data for 600636.SH
Downloaded data for 688239.SH
Downloaded data for 688622.SH
Downloaded data for 603897.SH
Downloaded data for 688656.SH


  0%|          | 24/7853 [00:00<02:58, 43.89it/s]

...

 10%|█         | 795/7853 [00:18<02:44, 42.91it/s]

Downloaded data for 603306.SH
Downloaded data for 603026.SH
Downloaded data for 688739.SH
Downloaded data for 688322.SH
Downloaded data for 603052.SH
Downloaded data for 688356.SH
Downloaded data for 600759.SH
Downloaded data for 600336.SH


 10%|█         | 801/7853 [00:18<02:45, 42.63it/s]


Downloaded data for 603866.SH
Downloaded data for 600234.SH
Downloaded data for 601033.SH
Downloaded data for 688220.SH
Downloaded data for 601999.SH
Downloaded data for 603150.SH


KeyboardInterrupt: 

In [ ]:
os.makedirs(
    "D:/qmt_datadir/logging/",
    exist_ok=True,
)
record_df.to_excel(
    f"D:/qmt_datadir/logging/{end_time}_record_{period}.xlsx",
    index=False,
    engine="openpyxl",
)